In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/Users/BadWizard/anaconda3/envs/theano/lib/python3.5/site-packages/matplotlib/style/core.py:197: UserWarning: In /Users/BadWizard/.matplotlib/stylelib/my_custom_style.mplstyle: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(message)


In [2]:
import pymc3 as pm3


# Example 1. Simple Linear regression with one independent variabes

In [3]:
N = 10000
x = 10 + 2*np.random.randn(N)
y = 5 + x + np.random.randn(N)
df = pd.DataFrame({'y':y, 'x':x})
df['constant'] = 1

df.head()

,x,y,constant
0,9.823447,13.437519,1
1,9.582009,15.131352,1
2,8.576958,14.864126,1
3,14.088832,18.801374,1
4,9.607870,14.195478,1


In [4]:
with pm3.Model() as model:
    b0 = pm3.Flat('constant',testval=5)
    b1 = pm3.Flat('x',testval=1)
    sigma = pm3.Flat('sigma',testval = 1)
    mu = b0 + b1 * df.x
    like = pm3.Normal('like',mu = mu, sd=sigma, observed=df.y)

In [5]:
with model:
    # obtain MLE 
    beta_mle = pm3.find_MAP()
    # obtain hessian at MAP
    hessian = pm3.approx_hessian(beta_mle)

Optimization terminated successfully.
         Current function value: 14157.159103
         Iterations: 6
         Function evaluations: 8
         Gradient evaluations: 8


In [6]:
se_1 = np.sqrt(np.diag(np.linalg.inv(hessian)))    
se = se_1[::-1]

In [7]:
se1

NameError: name 'se1' is not defined

In [ ]:
se

In [ ]:
beta_mle['x']

In [ ]:
b = np.array([beta_mle['constant'], beta_mle['x'], beta_mle['sigma']])

In [ ]:
results_pymc = pd.DataFrame({'parameters':b,'std err':se})
results_pymc.index=['Intercept','Slope','Sigma']   
results_pymc.head()

# Example 2. Multiple linear regression

In [ ]:
np.random.seed(314)
N = 100
alpha_real = 2.5
beta_real = [0.9, 1.5]
eps_real = np.random.normal(0, 0.5, size = N)

In [ ]:
X = np.array([np.random.normal(i, j, N) for i,j in zip([10, 2], [1, 1.5])])

In [ ]:
print('The mean of X is {}'.format(np.mean(X,axis=1)))
print('The std. of X is {}'.format(np.std(X,axis=1)))

In [ ]:
X_mean = X.mean(axis=1, keepdims=True)
X_mean

In [ ]:
X_centered = X - X_mean

In [ ]:
X_centered.mean(axis=1)

In [ ]:
X.T.dot(beta_real)

In [ ]:
np.dot(beta_real, X)

In [ ]:
y = alpha_real + np.dot(beta_real, X) + eps_real

In [ ]:
def scatter_plot(x, y):
    plt.figure(figsize=(10, 10))
    for idx, x_i in enumerate(x):
        plt.subplot(2,2,idx+1)
        plt.scatter(x_i, y)
        plt.xlabel("$x_{}$".format(idx), fontsize = 16)
        plt.ylabel("$y$", rotation=0, fontsize = 16)
        
    plt.subplot(2,2,idx+2)
    plt.scatter(x[0], x[1])
    plt.xlabel("$x_{}$".format(idx-1), fontsize = 16)
    plt.ylabel("$x_{}$".format(idx), rotation=0, fontsize = 16)
    
        
            

In [ ]:
scatter_plot(X, y)

In [ ]:
with pm3.Model() as model_mlr:
    alpha_tmp = pm3.Normal('alpha_tmp', mu=0, sd = 10)
    beta = pm3.Normal('$beta$', mu=0, sd = 1, shape = 2)
    epsilon = pm3.HalfCauchy('$epsilon$', 5)
    
    mu = alpha_tmp + pm3.math.dot(beta, X_centered)
    alpha = pm3.Deterministic('$alpha$', alpha_tmp - pm3.math.dot(beta, X_mean))
    y_pred = pm3.Normal('y_pred', mu=mu, sd=epsilon, observed = y)
    
    start = pm3.find_MAP()
    step = pm3.NUTS(scaling=start)
    trace_mlr = pm3.sample(5000, step=step, start=start )

In [ ]:
trace_mlr.varnames

In [ ]:
varnames = ['$alpha$','$beta$', '$epsilon$']
pm3.traceplot(trace_mlr, varnames)
;

In [ ]:
start['beta']